# Notebook 2: Option Trading with ML
## Feature Selection
What could possibly go wrong?
12/02/2020
Develop in this notebook. Leave D1 as the working demo until replaced by another fully working demo.

## Purpose of this Notebook - Select the Best Features from the Feature Set
In the first notebook in this series we created a set of features from stock price information and earnings report date. Do these features actually contain enough information for a Learner to predict profitable stock option trades? In this notebook we will evaluate the features using four techniques to determine the quality of each feature. Finally, you will choose the best features to use in training the Learners.

Good features can help a Learner perform well. Conversely, bad features can cause many problems for a Learner, such as over-fitting and long train and test times. Leakage of future information into a feature set can cause completely erroneous results and failure of the entire project.

## Contents
As discussed above, feature reduction and elimination are powerful tools to improve Learner performance. Many techniques are available, so consider these four as just a sample to get you started.

0. Scaling - Scaling is not really a feature reduction tool. Consider it more of a feature improvement tool. Learners prefer to work with features that have similar scale and distribution. This simplifies the mathematical analysis and reduces scaling errors. StandardScaler converts every feature to a common normal distribution - usually N(0,1). Feature scaling has many complexities, so read the docs to understand how it can affect your features, especially features with non-normal distributions.

1. SelectKBest - univariate feature selection and ranking. SelectKBest scores each feature individually, then ranks the features according to the scores.

2. Correlation with Heat Map - Using the pandas corr function and the Seaborn library, we can map the cross-correlations of each feature with every other feature and, most important, with the label.

3. RFE - Recursive Feature Elimination. RFE is a powerful function in the SKLearn library. It recursively tests groups of features together, gradually eliminating the weakest ones, until it arrives at the smallest ensemble of features that produces the best results. Read the docs to learn more about RFE.

4. Principal Component Analysis (PCA) - PCA is not really a feature reduction tool. It performs dimension reduction, by combining features into a few, typically two or three, artificial features by combining the original features mathematically to preserve the maximum independent variance. For problems with many dimensions, say hundreds or more, PCA can be very effective.

5. Shapley Adaptive Model SHAP

- What You Need: Basic knowledge of Python, pandas, numpy, dataframes, and statistics.

- What You Don't Need: Knowledge of linear algebra, calculus, or Machine Learning. The Python libraries, particularly scipy and SKLearn for ML, do all the heavy lifting.

You Will Learn:

- How to scale a feature set with StandardScaler.
- Three feature selection methods.
- Principal Component Analysis (PCA)
- a few more capabilities of MatPlotlib


## StandardScaler and SelectKBest

Evaluation Questions
- What is the importance of feature scaling?
- Before you look at the results, choose the three features that you rank highest.
- How do your choices compare with the ranking of SelectKBest? Any surprises? Why?
- Given the scores from SelectKBest, how many features woyuld you choose for the training set? Why?

Code Questions
- What is the distribution of the features after running StandardScaler? Print a few to confirm?
- What happens to the mean and standard deviation values of each original feature? Print a few.
- How does SelectKBest calculate the feature score?
- Create a dataframe to collect the scores and ranking for each feature selection algorithm. 
- Use this dataframe to compare the results at the end of this notebook.
- The features ER - Expiry and 2 day Velocity show NaNs as scores. Why? Can you change these scores?

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing     import StandardScaler
from sklearn.decomposition     import PCA

raw_features = pd.read_csv('opt_sample.csv', index_col = 0)

# Remove duplicate features
dup_features = ['STO_open_strike_delta%', 'stock_price_at_open']
print(raw_features.shape)
raw_features_headers = raw_features.columns.to_list() 
print(raw_features_headers)
features_np = raw_features.to_numpy(copy = True)
scaler = StandardScaler()
features = scaler.fit_transform(features_np)

X_train = features[:,:28]  #independent features (columns)
y_train = features[:,-1]    #label column

# Apply SelectKBest class to measure and rank features.

bestfeatures = SelectKBest(score_func=f_classif, k='all')
fit = bestfeatures.fit(X_train,y_train)
feature_scores = pd.DataFrame(fit.scores_)
feature_columns = pd.DataFrame(raw_features.columns)

feature_scores = pd.concat([feature_scores, feature_columns],axis=1)
feature_scores.columns = ['KBest Score', 'Feature']
feature_scores.sort_values(['KBest Score'], ascending = False, inplace = True)
feature_scores = feature_scores.reset_index(drop = True)
feature_scores['KBest Score'] = round(feature_scores['KBest Score']) 
feature_scores['KBest_rank'] = feature_scores.index
print(feature_scores)


(5577, 29)
['stock_price_at_STO_offer', 'STO_strike', 'STO_days_long', 'STO_offer_ARR', 'stock_price_at_open', 'STO_ask_fee%', 'STO_offer_strike_delta%', '45 day SMA', '30 day SMA', '15 day SMA', '10 day SMA', 'Yearly Min', 'Yearly Max', '1 days old', '2 days old', '3 days old', '4 days old', '5 days old', 'RSI', 'MFI', 'MACD', '45 day vel', '30 day vel', '15 day vel', '10 day vel', '5 day vel', '2 day vel', 'ER - Expiry', 'label']
    KBest Score                   Feature  KBest_rank
0        1355.0                30 day SMA           0
1        1231.0                45 day SMA           1
2        1010.0                STO_strike           2
3         850.0                15 day SMA           3
4         750.0                10 day SMA           4
5         744.0                3 days old           5
6         738.0                4 days old           6
7         696.0   STO_offer_strike_delta%           7
8         670.0                5 days old           8
9         665.0         

## Cross-correlation and Seaborn Heat Map
Evaluation questions
- What row shows the cross-correlation of each feature to the labell?
- Why is the correlation value exactly '1' for every cell on the diagonal?
- What is the difference between +1 and -1 in correlation? Which has more information? Why? Why not?

Code Questions
- Run the correlation heat map with the unscaled features. What differences, and why?
- Record the feature rankings in a column in your feature rank dataframe. Be sure to label it 'Cross-correlation.'


In [2]:
# Seaborn heat map
import seaborn as sb
import matplotlib.pyplot as plt

#print(repr(feature_columns))
feature_col_list = feature_columns[0].to_list()

scaled_features = pd.DataFrame(features, columns = feature_col_list)
plt.figure(figsize=(17,17))
#plot heat map
#g=sb.heatmap(scaled_features.corr(),annot=True,cmap="RdYlGn")

corr_matrix = scaled_features.corr(method = 'pearson')
# Rename index col to 'feature name'. Then sort and reindex.
# Set label corr to -1.0. This is a clumsy method to get the sort to work correctly.
# Find better approach later (delete the label row?)
corr_matrix.loc[corr_matrix['label'] == 1.0] = -1.0

corr_matrix['feature'] = corr_matrix.index
corr_matrix.sort_values(by = 'label', axis = 0, ascending = False, ignore_index = True, inplace = True)
corr_matrix['corr_rank'] = corr_matrix.index
print_cols = ['feature', 'corr_rank', 'label']
print(corr_matrix[print_cols])
# Sort corr_matrix and featrure_scores by feature name to get correct alignment before copying cols to feature_scores.
# Can do this by concat also.
corr_matrix.sort_values(by = 'feature', axis = 0, ascending = True, ignore_index = True, inplace = True)
feature_scores.sort_values(by = 'Feature', axis = 0, ascending = True, ignore_index = True, inplace = True)

feature_scores[print_cols] = corr_matrix[print_cols]
print(feature_scores)


                     feature  corr_rank     label
0                 30 day SMA          0  0.442180
1                 45 day SMA          1  0.425345
2                 STO_strike          2  0.391565
3                 15 day SMA          3  0.363756
4                 10 day SMA          4  0.344375
5                 3 days old          5  0.343056
6                 4 days old          6  0.341922
7    STO_offer_strike_delta%          7  0.333154
8                 5 days old          8  0.327573
9                 2 days old          9  0.326405
10                1 days old         10  0.312986
11       stock_price_at_open         11  0.307195
12  stock_price_at_STO_offer         12  0.302931
13                45 day vel         13  0.264699
14                Yearly Min         14  0.185214
15             STO_days_long         15  0.158391
16                      MACD         16  0.148949
17                30 day vel         17  0.039534
18                       RSI         18  0.009135


<Figure size 1224x1224 with 0 Axes>

## RFE
The Recursive Feature Eliminator (RFE) compares many sets of features to find the smallest set with the best results. Read the docs, then consider these questions.

Evaluation Questions
- How does RFE work?
- What is the estimator. What does it do?

Code Questions
- Describe the arguments given to the RFE selector.
- Time the execution of RFE selector. Why does it take so long to run?
- Would this execution time typically be a problem in a real-world application? Why, or not?
- Write a function to run RFE with a range of values. Any differences in the results? Why?
- Record the results of RFE ranking for the features in your feature ranking dataframe.

In [3]:
# Recursive Feature Elimination - continuous model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

#estimator = SVR(kernel="linear")
#selector = RFE(estimator, n_features_to_select = 1, step=1, verbose = 2)
#print('starting selector.')
#selector = selector.fit(X_train, y_train)
#print('finished selector.')
#print(selector.support_)   # selected features
#print(selector.ranking_) # feature rank
#RFE_rank = np.transpose(RFE_rank).copy()

# SVR feature rank shown below.
#RFE_rank = [9, 8, 27, 21, 24, 26, 22, 14, 10, 3, 1, 6, 7, 18, 20, 23, 12, 19, 15, 13, 2, 4, 5, 11, 16, 17, 28, 25]
#RFE_rank.append(len(RFE_rank)+1)

# RFE - Classifier version

from sklearn.ensemble import RandomForestClassifier
from xgboost          import XGBClassifier
import catboost
from catboost         import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_train, y_train)
RFE_ET_rank = clf.feature_importances_
print(RF_ET_rank)

stop

#clf = RandomForestClassifier(random_state = 97, verbose = 1)
#clf = clf.fit(X_train, y_train)
#RFE_RF_rank = clf.feature_importances_
#print(RFE_RF_rank)

RFE_feature_df = pd.DataFrame(RFE_rank, columns = ['RFE_rank'])
RFE_feature_df['RFE_feature'] = raw_features_headers
print(RFE_feature_df)
RFE_feature_df.sort_values(by = 'RFE_feature', axis = 0, ascending = True, ignore_index = True, inplace = True)
RFE_copy_cols = ['RFE_rank', 'RFE_feature']
feature_scores[RFE_copy_cols] = RFE_feature_df[RFE_copy_cols]
print(feature_scores)
feature_scores.to_csv('feature_scores.csv')

ValueError: Unknown label type: 'continuous'

For feature ranking, add Permutation Importance and Shapley Addative (SHAP)


In [ ]:
# from sklearn.inspection import permutation_importance
# model = Ridge(alpha=1e-2).fit(X_train, y_train)
# model.score(X_val, y_val)

# r = permutation_importance(model, X_val, y_val,
#                           n_repeats=30,
#                           random_state=0)

# for i in r.importances_mean.argsort()[::-1]:
#    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#        print(f"{diabetes.feature_names[i]:<8}"
#              f"{r.importances_mean[i]:.3f}"
#              f" +/- {r.importances_std[i]:.3f}")

import shap
#explainer = shap.TreeExplainer(model)
#shap_values = explainer.shap_values(X_train)
#shap.summary_plot(shap_values, X_train, plot_type="bar")

## PCA

Principal Component Analysis (PCA) extracts the non-correlated information in each given feature and systhesizes a specified number of orthogonal new ((artificial) components. Thus, it performs significant dimensionality reduction.

Evalution Questions
- How is PCA different from feature ranking?
- What are some benefits of dimensionality reduction?
- What are some drawbacks of PCA?
- Describe explained variance (ev), and explained variance ratio (evr).
- Woud you consider the ev and evr shown to be adequate? Why, or not?

Code Questions
- Run PCA with different arguments. What arguments produce better results?
- Run PCA with non-scaled features. Any differences?

In [ ]:
from sklearn.decomposition     import PCA

X_reduced = PCA(n_components = 2).fit(scaled_features)
print(X_reduced.components_)
print(X_reduced.explained_variance_)
print(X_reduced.explained_variance_ratio_)


## Latent Dirichlet Allocation (LDA)

Like PCA, LDA synthesizes artificial features (components) based on the non-correlated information contained in each of the given featuures. However, unlike PCA, LDA considers the labels, so it maximizes the separation of data clusters along label boundaries. PCA, which does not consider labels, simply identifies clusters and maximizes the separation of the clusters themselves.

Evaluation Questions
- Is LDA supervised or unsupervised?  PCA? 
- Which one do you expect to perform better for this problem? Why, or not?
- How will you measure and evaluate their comparative performance?

Code Questions
- Create scatterplots of PCA and LDA. What insight do these provide?
- Use the Learner function in Notebook 3 to run a set of Learners with PCA and LDA features. 
- What differences in Learning performance and train/test execution times? Why?

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components = 2)
transformed = lda.fit_transform(X_train, y_train)


## Choose Your Features
Now you have all the information you need to select the features you will use to train the Learners. 

Evaluation Questions
- Did any of the rankings surprise you? Which ones? Why?
- Each feature selector generated a different ranking for the features. Why?
- Describe comparative strengths and weaknesses of each feature selector. 
- Which feature selector do you prefer? Why?
- Would you combine the rankings to get a better overall feature rank? How?  Why, or not?
- What is your ranking for the features? Why?

Code Questions
- Compare the rankings of each feature selector by graphing the scores with MatplotLib. 
- Write a function to do grouped bar charts, starting with the function in the next notebook that displays grouped bar charts. - How can you display the information in the most effective way to showcase your selection?


I am eager to hear of your adventures in learning ML and especially in your challenges and results in tackling your own applications for ML. Please email me with questions, comments, and results of your work. 

I love to hear stories about experiences in ML, so give me a shout.

Best wishes for selecting powerful features!

Richard Barrett

richard at rbshomes dot com



